In [4]:
"""
FastF1 Lap-Level Data Fetcher for HPC F1 AI Strategy System

Downloads lap-aggregated telemetry and race data from a specific F1 session.
Creates per-lap statistics instead of per-time-sample data.

Output columns:
- lap_number: The lap number
- total_laps: Total laps in the race
- lap_time: Time taken for this lap
- average_speed: Average speed during the lap (km/h)
- max_speed: Maximum speed during the lap (km/h)
- tire_compound: Tire compound used
- tire_life_laps: Number of laps on current tires
- track_temperature: Average track temperature during the lap
- rainfall: Whether it rained during the lap
"""
import fastf1
import pandas as pd
import numpy as np

In [5]:
# 1. Load the session
session = fastf1.get_session(2023, 'Monza', 'R')
session.load(telemetry=True, laps=True, weather=True)

# 2. Pick the driver
driver_laps = session.laps.pick_drivers('ALO')

# Get total number of laps in the race (maximum lap number from all drivers)
total_laps = session.laps['LapNumber'].max()

print(f"Loaded session: 2023 Monza Race")
print(f"Driver: ALO (Alonso)")
print(f"Total laps in race: {total_laps}")
print(f"Driver laps: {len(driver_laps)}")

core           INFO 	Loading data for Italian Grand Prix - Race [v3.6.1]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core        WARNING 	Driver 1 completed the race distance 06:25.888000 before the recorded end of the session.
core        WARNING 	Driver 11 completed the race distance 06:19.824000 before the recorded end of the session.
core        WAR

Loaded session: 2023 Monza Race
Driver: ALO (Alonso)
Total laps in race: 51.0
Driver laps: 51


In [6]:
# 3. Get weather data for merging
weather = session.weather_data
weather['SessionTime'] = pd.to_timedelta(weather['Time'])

# 4. Get all laps for position calculation
all_laps = session.laps

# 5. Create lap-level data by aggregating telemetry
lap_data_list = []

for lap_idx in driver_laps.index:
    lap = driver_laps.loc[lap_idx]
    lap_number = lap['LapNumber']
    
    # Skip invalid laps
    if pd.isna(lap_number):
        continue
    
    # Get telemetry for this lap
    car_data = lap.get_car_data()
    
    if car_data is not None and len(car_data) > 0:
        # Calculate speed statistics
        avg_speed = car_data['Speed'].mean()
        max_speed = car_data['Speed'].max()
        
        # Get lap time
        lap_time = lap['LapTime']
        
        # Get tire information
        tire_compound = lap['Compound']
        tire_life = lap['TyreLife']
        
        # Get position data for this lap
        position = lap['Position']
        
        # Calculate gaps: get all drivers' data for this lap
        this_lap_all_drivers = all_laps[all_laps['LapNumber'] == lap_number].copy()
        
        # Sort by position to calculate gaps
        this_lap_all_drivers = this_lap_all_drivers.sort_values('Position')
        
        # Calculate cumulative race time for gap calculations
        gap_to_leader = 0.0
        gap_to_ahead = 0.0
        
        if pd.notna(position) and position > 1:
            # Get our driver's data
            our_data = this_lap_all_drivers[this_lap_all_drivers['Driver'] == 'ALO']
            
            if not our_data.empty:
                # Time at the end of this lap (cumulative)
                # Note: FastF1 provides 'Time' which is cumulative race time
                our_time = our_data['Time'].values[0]
                
                # Get leader's time (P1)
                leader_data = this_lap_all_drivers[this_lap_all_drivers['Position'] == 1]
                if not leader_data.empty:
                    leader_time = leader_data['Time'].values[0]
                    # Convert numpy.timedelta64 to seconds
                    gap_to_leader = (our_time - leader_time) / np.timedelta64(1, 's')
                
                # Get car ahead's time (Position - 1)
                ahead_position = position - 1
                ahead_data = this_lap_all_drivers[this_lap_all_drivers['Position'] == ahead_position]
                if not ahead_data.empty:
                    ahead_time = ahead_data['Time'].values[0]
                    # Convert numpy.timedelta64 to seconds
                    gap_to_ahead = (our_time - ahead_time) / np.timedelta64(1, 's')
        
        # Get weather data for this lap (use lap start time)
        lap_start_time = pd.to_timedelta(lap['LapStartTime'])
        
        # Find closest weather data
        weather_at_lap = weather.iloc[(weather['SessionTime'] - lap_start_time).abs().argmin()]
        track_temp = weather_at_lap['TrackTemp']
        rainfall = weather_at_lap['Rainfall']
        
        # Create lap record
        lap_record = {
            'lap_number': int(lap_number),
            'total_laps': int(total_laps),
            'position': int(position) if pd.notna(position) else None,
            'gap_to_leader': round(gap_to_leader, 3) if gap_to_leader > 0 else 0.0,
            'gap_to_ahead': round(gap_to_ahead, 3) if gap_to_ahead > 0 else 0.0,
            'lap_time': lap_time,
            'average_speed': round(avg_speed, 2),
            'max_speed': round(max_speed, 2),
            'tire_compound': tire_compound,
            'tire_life_laps': int(tire_life) if pd.notna(tire_life) else None,
            'track_temperature': round(track_temp, 2) if pd.notna(track_temp) else None,
            'rainfall': bool(rainfall)
        }
        
        lap_data_list.append(lap_record)
    
    # Progress indicator
    if lap_number % 10 == 0:
        print(f"Processed lap {int(lap_number)}...")

# 6. Create final dataframe
laps_df = pd.DataFrame(lap_data_list)

print(f"\n✓ Created lap-level dataframe with {len(laps_df)} laps")
print(f"Laps covered: {laps_df['lap_number'].min()} to {laps_df['lap_number'].max()}")
laps_df.head(10)

Processed lap 10...
Processed lap 20...
Processed lap 30...
Processed lap 40...
Processed lap 50...

✓ Created lap-level dataframe with 51 laps
Laps covered: 1 to 51


,lap_number,total_laps,position,gap_to_leader,gap_to_ahead,lap_time,average_speed,max_speed,tire_compound,tire_life_laps,track_temperature,rainfall
0,1,51,11,6.223,0.620,0 days 00:01:33.340000,210.17,326.0,MEDIUM,1,42.5,False
1,2,51,11,8.229,0.712,0 days 00:01:28.012000,236.87,330.0,MEDIUM,2,42.5,False
2,3,51,11,9.799,0.898,0 days 00:01:27.546000,236.40,331.0,MEDIUM,3,43.2,False
3,4,51,11,10.953,0.919,0 days 00:01:27.221000,240.13,341.0,MEDIUM,4,43.2,False
4,5,51,11,11.563,0.917,0 days 00:01:27.033000,236.09,345.0,MEDIUM,5,43.1,False
5,6,51,11,12.123,0.923,0 days 00:01:27.175000,236.74,343.0,MEDIUM,6,43.3,False
6,7,51,11,12.694,0.387,0 days 00:01:26.929000,239.72,340.0,MEDIUM,7,43.6,False
7,8,51,10,13.413,2.760,0 days 00:01:26.943000,238.45,351.0,MEDIUM,8,43.6,False
8,9,51,10,14.320,3.387,0 days 00:01:27.383000,236.81,330.0,MEDIUM,9,43.6,False
9,10,51,10,15.177,3.760,0 days 00:01:27.368000,232.42,331.0,MEDIUM,10,43.9,False


In [7]:
# Display dataframe info and statistics
print("Lap-Level Dataframe Info:")
print(f"Total laps: {len(laps_df)}")
print(f"\nColumn types:")
print(laps_df.dtypes)
print(f"\nBasic statistics:")
laps_df.describe()

Lap-Level Dataframe Info:
Total laps: 51

Column types:
lap_number                     int64
total_laps                     int64
position                       int64
gap_to_leader                float64
gap_to_ahead                 float64
lap_time             timedelta64[ns]
average_speed                float64
max_speed                    float64
tire_compound                 object
tire_life_laps                 int64
track_temperature            float64
rainfall                        bool
dtype: object

Basic statistics:


,lap_number,total_laps,position,gap_to_leader,gap_to_ahead,lap_time,average_speed,max_speed,tire_life_laps,track_temperature
count,51.000000,51.0,51.000000,51.000000,51.000000,51,51.000000,51.000000,51.000000,51.000000
mean,26.000000,51.0,9.803922,29.418314,2.889235,0 days 00:01:27.596803921,235.797059,333.686275,15.411765,42.898039
std,14.866069,0.0,1.058671,13.842909,1.707825,0 days 00:00:03.069690434,7.855085,4.921342,8.616673,0.876924
min,1.000000,51.0,6.000000,6.223000,0.387000,0 days 00:01:26.105000,191.140000,322.000000,1.000000,40.800000
25%,13.500000,51.0,9.000000,16.928500,1.123000,0 days 00:01:26.715000,236.105000,331.000000,8.500000,42.500000
50%,26.000000,51.0,10.000000,29.113000,2.799000,0 days 00:01:26.943000,237.130000,332.000000,15.000000,43.100000
75%,38.500000,51.0,10.000000,43.405500,4.136500,0 days 00:01:27.328500,238.655000,334.000000,21.000000,43.600000
max,51.000000,51.0,12.000000,48.171000,8.135000,0 days 00:01:47.272000,241.700000,351.000000,33.000000,44.300000


In [8]:
# Save to CSV
laps_df.to_csv("scripts/ALONSO_2023_MONZA_LAPS.csv", index=False)
print("✓ Saved lap-level data to scripts/ALONSO_2023_MONZA_LAPS.csv")

✓ Saved lap-level data to scripts/ALONSO_2023_MONZA_LAPS.csv


In [9]:
# Display position and gap information
print("\nPosition and Gap Analysis:")
print(f"Starting position: P{laps_df['position'].iloc[0]}")
print(f"Final position: P{laps_df['position'].iloc[-1]}")
print(f"Average gap to leader: {laps_df['gap_to_leader'].mean():.2f}s")
print(f"Average gap to car ahead: {laps_df['gap_to_ahead'].mean():.2f}s")
print(f"\nPosition changes over race:")
print(laps_df[['lap_number', 'position', 'gap_to_leader', 'gap_to_ahead']].head(15))


Position and Gap Analysis:
Starting position: P11
Final position: P9
Average gap to leader: 29.42s
Average gap to car ahead: 2.89s

Position changes over race:
    lap_number  position  gap_to_leader  gap_to_ahead
0            1        11          6.223         0.620
1            2        11          8.229         0.712
2            3        11          9.799         0.898
3            4        11         10.953         0.919
4            5        11         11.563         0.917
5            6        11         12.123         0.923
6            7        11         12.694         0.387
7            8        10         13.413         2.760
8            9        10         14.320         3.387
9           10        10         15.177         3.760
10          11        10         15.829         3.984
11          12        10         16.760         4.129
12          13        10         17.031         3.995
13          14        10         17.666         4.076
14          15        10     

In [10]:
# Show tire strategy
print("\nTire Strategy:")
tire_stints = laps_df.groupby(['tire_compound']).agg({
    'lap_number': ['min', 'max', 'count'],
    'average_speed': 'mean',
    'tire_life_laps': 'max'
}).round(2)
tire_stints.columns = ['First Lap', 'Last Lap', 'Laps on Compound', 'Avg Speed', 'Max Tire Life']
print(tire_stints)


Tire Strategy:
               First Lap  Last Lap  Laps on Compound  Avg Speed  Max Tire Life
tire_compound                                                                 
HARD                  22        51                30     236.42             33
MEDIUM                 1        21                21     234.91             21
